In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from tqdm.keras import TqdmCallback
import os
import json

In [2]:
"""Выборка для обучения получена из работы
Arrhenius Crossover Temperature of Glass-Forming Liquids Predicted by an Artificial Neural Network """
# Подготовленные данные в виде Excel таблицы, в случае отсутсвия можете получить по ссылке
# https://disk.yandex.ru/i/JURRL-BjFInPUg
# Дополнительная информация:
# В таблице 91 запись, из них 57 - с известными экспериментальными T_A_empirical, остальные без (NaN)
# Обязательные столбцы при составлении выборки: ['T_g','T_m','T_A_empirical','m']
try:
    excel_import = pd.read_excel('..\\data\\input_data.xlsx')
except FileNotFoundError:
    print(f'В папку с NoteBook\'ом добавьте файл input_data.xlsx.\n'
          f'Скачать можно по ссылке https://disk.yandex.ru/i/JURRL-BjFInPUg')
    raise FileNotFoundError
excel_import.loc[:, 'T_g/T_m'] = excel_import.loc[:, 'T_g'] / excel_import.loc[:, 'T_m']

In [3]:
# Конфигурация нейронки
BATCH_SIZE = 128  # размер пакета 128>57, SGD переходит в обычный градиентный спуск
TRAIN_RATE = 0.1
EPOCHS = 1000

In [4]:
## После нахождения оптимального ввода поищем оптимальное число нейронов


def get_model(neuron_count=10):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(2,)),
        tf.keras.layers.Dense(neuron_count, activation='tanh'),
        tf.keras.layers.Dense(neuron_count, activation='tanh'),
        tf.keras.layers.Dense(1),
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate=TRAIN_RATE)
    model.compile(optimizer=optimizer,
                  loss='mse')
    return model

In [5]:
train_data = excel_import[excel_import.T_A_empirical.notna()].loc[:,
             ['T_g', 'T_m', 'T_g/T_m', 'm', 'T_A_empirical']]
## Данные, которые с известным экспериментальным значением, поделим на 10000, чтобы значения были в [0,1]

target_normalizer = tf.keras.layers.Normalization(axis=1)
target_normalizer.adapt(train_data['T_A_empirical'])

target_denormalizer = tf.keras.layers.Normalization(axis=1, invert=True)
target_denormalizer.adapt(train_data['T_A_empirical'])

train_data

,T_g,T_m,T_g/T_m,m,T_A_empirical
34,995,1664,0.597957,59.0,1839.0
35,1087,1393,0.780330,24.0,1503.0
36,1113,1825,0.609863,54.0,2010.0
37,530,723,0.733057,36.0,1073.0
38,208,321,0.647975,125.0,328.0
39,179,238,0.752101,75.0,241.0
40,185,270,0.685185,70.0,262.0
41,195,275,0.709091,70.0,261.0
42,187,223,0.838565,60.0,251.0
43,196,234,0.837607,66.0,268.0


In [6]:
train_target = tf.transpose(target_normalizer(train_data['T_A_empirical']))
input_normalizer = tf.keras.layers.Normalization()
input_normalizer.adapt(train_data[['T_g','T_m']])
train_input = input_normalizer(train_data[['T_g','T_m']])

MSE = pd.DataFrame([])
to_do = True
if to_do:
    for neuron_count in range(1,26):
        model = get_model(neuron_count)
        history = model.fit(train_input, train_target,
                  verbose=0,  # отключение вывода tf
                  callbacks=[TqdmCallback(verbose=0)],
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS)  # вывод с помощью TQDM
        model.save(f'models\\series_neuron\\model_{neuron_count}')
        MSE[f'MSE_{neuron_count}'] = history.history['loss']

0epoch [00:00, ?epoch/s]

INFO:tensorflow:Assets written to: models\series_neuron\model_1\assets


INFO:tensorflow:Assets written to: models\series_neuron\model_1\assets


0epoch [00:00, ?epoch/s]

INFO:tensorflow:Assets written to: models\series_neuron\model_2\assets


...

INFO:tensorflow:Assets written to: models\series_neuron\model_24\assets


INFO:tensorflow:Assets written to: models\series_neuron\model_24\assets


0epoch [00:00, ?epoch/s]

INFO:tensorflow:Assets written to: models\series_neuron\model_25\assets


INFO:tensorflow:Assets written to: models\series_neuron\model_25\assets


In [7]:
%matplotlib tk
MSE_array = np.array(MSE)

fig, ax = plt.subplots(figsize=(12, 10),
                       ncols=5,nrows=5,)
for i in range(MSE_array.shape[1]):
    j = i % 5
    k = int(i/5)
    ax[j,k].plot(np.linspace(0,1000,1000),MSE_array[:,i])
    ax[j, k].set_ylabel('MSE')
    ax[j, k].set_xlabel('Epoch')
    ax[j,k].set_title(f'Neuron count = {i+1}')
fig.savefig('..\\graphics\\MSE_from_neuron')
fig.tight_layout()
plt.show(block=False)



In [20]:
# подсчет MSE для каждой модели на всей выборке
model_folders = os.listdir('models\\series_neuron')
for i, folder in enumerate(model_folders):
    model = tf.keras.models.load_model(f'models\\series_neuron\\{folder}')
    predict = target_denormalizer(model.predict(train_input))
    table = pd.DataFrame(train_data['T_A_empirical'])
    table['T_A_predicted'] = predict
    table['difference'] = (table['T_A_empirical'] - table['T_A_predicted']).abs()
    table.to_csv(f'..\\data\\neuron_series_data\\model_{i}_data.csv')


2/2 [==============================] - 0s 3ms/step


In [22]:
data_folders = os.listdir('..\\data\\neuron_series_data')
len(data_folders)

29

In [29]:
%matplotlib tk

fig, axs = plt.subplots(figsize=(15,15),ncols=5,nrows=5)
fig.suptitle('Температуры Аррениуса от количества нейронов')
mse_table = np.array([])
max_difference = np.array([])
for i, data in enumerate(data_folders[:25]):
    table = pd.read_csv(f'..\\data\\neuron_series_data\\{data}')
    mse = ((table['difference']**2).sum() / table['difference'].shape[0]) ** (1 / 2)
    mse_table = np.append(mse_table, mse)
    max_difference = np.append(max_difference, table['difference'].max())
    j = i % 5
    k = int(i / 5)
    axs[j, k].plot(table['T_A_predicted'], table['T_A_empirical'], '.r')
    axs[j, k].plot([0, 2250], [0, 2250], '-k')
    axs[j, k].set_ylabel('$T_{emp}, K$')
    axs[j, k].set_xlabel('$T_{pred}, K$')
    axs[j,k].text(1000,0,f'mse = {np.round(mse,2)}\nmax_diff = {np.round(max_difference[i],2)}')
    axs[j, k].set_title(f'{i} neurons')
print(mse_table)
print(max_difference)
fig.tight_layout()
fig.show()
fig.savefig('..\\graphics\\Temp_from_neuron')

[68.22967578 50.28530062 54.96740036 50.14691705 51.64387971 51.69728993
 50.88829773 51.29037207 52.02256485 49.57594973 50.72268043 50.23106379
 50.09134934 45.92422279 50.03328294 45.02151303 48.3637124  37.84170984
 35.71822789 35.07486467 34.44425807 46.36867235 49.39604219 49.08167538
 99.55396884]
[259.20239258 233.76647949 247.06848145 234.08129883 249.65985107
 243.75372314 239.5826416  246.98419189 243.83984375 234.94647217
 246.93005371 234.38439941 239.29980469 193.75750732 244.70825195
 167.48522949 224.66064453 148.88366699 105.29602051 112.39587402
 110.87084961 196.36175537 229.02130127 227.17181396 239.7277832 ]


In [31]:
plt.plot(np.linspace(0,len(mse_table),len(mse_table)),mse_table)
plt.show()

In [36]:

n = len(mse_table)
plt.plot(np.linspace(0,n,n),mse_table)
plt.ylabel('MSE')
plt.xlabel('neuron count')
plt.title("Зависимость MSE от количества нейронов в слое")
plt.savefig('..\\graphics\\MSE_from_neuron_dependence')